In [ ]:
import os.path
import glob
import multiprocessing

import numpy as np
import pandas as pd
import networkx as nx

In [ ]:
""" Named Constants used for addressing the indexes of model Node States """
NS_STATE = 0
NS_NEXT_STATE = 1
NS_WAS_ACTIVATED_PREV_STEP = 2
NS_INIT_STATE = 3
NS_INIT_WAS_ACTIVATED_PREV_STEP = 4

# """ Reads the network from the given inNeighbors.json file """
# def read_network(in_network_file):
#     node_in_neighbors = ReadNeighborsAsIntegers(in_network_file)
#     name_parts = os.path.basename(in_network_file).split('_')
#     #print(name_parts)
#     network_index = int(name_parts[1])
#     network_size = int(name_parts[2][1:])
#     beta = float((name_parts[3][1:]).replace('o','.'))
#     din = float((name_parts[4][3:]).replace('o','.'))
#     dout = float((name_parts[5][4:]).replace('o','.'))
#     net = {"id" : network_index,
#            "size": network_size,
#            "beta": beta,
#            "delta_in": din,
#            "delta_out": dout,
#            "in_neighbors": node_in_neighbors,
#            }
#     return net

""" Given a network data read from a file, creates and initializes a usable simulation instance for DOI experiments
    Susceptible == False
    Infected == True
    Returns created simulation dictionary object
"""
def setup_simulation_instance(in_network, in_init_fract_infection):
    network_size = in_network["size"]
    NodeState = np.zeros((5, network_size), dtype=bool)
    init_infected_nodeidx_list = np.random.choice(network_size, round(in_init_fract_infection * network_size), replace=False)
    NodeState[NS_INIT_STATE, init_infected_nodeidx_list] = True
    NodeState[NS_INIT_WAS_ACTIVATED_PREV_STEP, init_infected_nodeidx_list] = True
    #print(f"Initial infected nodes: {len(init_infected_nodeidx_list)}")
    sim = {"NodeState": NodeState, "InitInfects":init_infected_nodeidx_list}
    return sim
    
""" Copies initial states to the current states of nodes """
def initialize(in_network, inout_simulation):
    NodeState = inout_simulation["NodeState"]
    NodeState[NS_STATE] = NodeState[NS_INIT_STATE]
    NodeState[NS_NEXT_STATE] = NodeState[NS_INIT_STATE]
    NodeState[NS_WAS_ACTIVATED_PREV_STEP] = NodeState[NS_INIT_WAS_ACTIVATED_PREV_STEP]
    
   
""" Runs the simulation """
def run(inout_simulation, in_network, in_model, in_max_steps=10000):
    
    infection_at_step = []
    
    network_size = in_network["size"]
    NodeState = inout_simulation["NodeState"]
    
    in_model.setup(in_network, inout_simulation)
    
    for step in (range(in_max_steps)):
        
        infection_at_this_step = 0

        # Process F > G for all Susceptible nodes and mark for state change
        for node_idx in range(network_size):
            if not NodeState[NS_STATE, node_idx]:
                # execute following for all Susceptible
                F = in_model.F(node_idx, NodeState, in_network["in_neighbors"], inout_simulation)
                G = in_model.G()
                if G <= F:
                    NodeState[NS_NEXT_STATE, node_idx] = True
        
        # Reset excitement of all nodes back
        NodeState[NS_WAS_ACTIVATED_PREV_STEP] = False
        
        for node_idx in range(network_size):
            if not NodeState[NS_STATE, node_idx] and NodeState[NS_NEXT_STATE, node_idx]:
                NodeState[NS_WAS_ACTIVATED_PREV_STEP, node_idx] = True
                infection_at_this_step += 1
        
        # Copy the next state onto state
        NodeState[NS_STATE] = NodeState[NS_NEXT_STATE]

        infection_at_step.append( infection_at_this_step )
        
        # stop if no change for 1000 steps
        if step > 1000 and step % 1000 == 1 and np.sum(infection_at_step[-1000:]) == 0:
            #print("No change for 1000 steps")
            break

    inout_simulation["InfectionAtStep"] = infection_at_step
    inout_simulation["StopAt"] = step
    return

In [ ]:
class LATM:
    def __init__(self, in_threshold):
        self.threshold = in_threshold
        
    def setup(self, in_network, inout_simulation):
        pass
        
    def F(self, in_node_idx, in_NodeState, in_inNeighbors, inout_simulation):
        neighborhood = in_inNeighbors[in_node_idx] if in_node_idx in in_inNeighbors else []
        infected_neighborhood = [neighbor_idx for neighbor_idx in neighborhood if in_NodeState[NS_STATE, neighbor_idx]]
        return len(infected_neighborhood)

    def G(self):
        return self.threshold

In [ ]:
class LFTM:
    def __init__(self, in_threshold):
        self.threshold = in_threshold
        
    def setup(self, in_network, inout_simulation):
        pass
        
    def F(self, in_node_idx, in_NodeState, in_inNeighbors, inout_simulation):
        neighborhood = in_inNeighbors[in_node_idx] if in_node_idx in in_inNeighbors else []
        infected_neighborhood = [neighbor_idx for neighbor_idx in neighborhood if in_NodeState[NS_STATE, neighbor_idx]]
        return len(infected_neighborhood) / len(neighborhood) if len(neighborhood) > 0 else 0

    def G(self):
        return self.threshold

In [ ]:
class ICM:
    def __init__(self, in_imitation):
        self.imitation = in_imitation
        
    def setup(self, in_network, inout_simulation):
        pass
        
    def F(self, in_node_idx, in_NodeState, in_inNeighbors, inout_simulation):
        neighborhood = in_inNeighbors[in_node_idx] if in_node_idx in in_inNeighbors else []
        active_neighborhood = [neighbor_idx for neighbor_idx in neighborhood if in_NodeState[NS_WAS_ACTIVATED_PREV_STEP, neighbor_idx]]
        return 1.0 - ((1.0 - self.imitation) ** len(active_neighborhood))

    def G(self):
        return np.random.rand()

In [ ]:
def run_dynamic_trial(in_network, in_model, in_initial_frac_infection):
    # setup 
    sim = setup_simulation_instance(in_network, in_initial_frac_infection)

    initialize(in_network, sim)
    #print(sim['NodeState'])

    run(sim, in_network, in_model)

    #print("Stopped at : " + str(sim["StopAt"]))
    #print(sim['NodeState'])
    
    final_infected_fraction = (sim['NodeState'][NS_STATE].sum() - len(sim["InitInfects"])) / in_network['size']
    net_percent_value = np.sum([v * (0.9 ** t) for t,v in enumerate(sim["InfectionAtStep"])])
    return net_percent_value, final_infected_fraction

In [ ]:
def run_model(in_network, in_model_name, in_model_param, in_initial_frac_infection):
    np.random.seed() # <-- multiprocessing with different seeds
    if "LATM" == in_model_name:
        return run_dynamic_trial(in_network, LATM(in_model_param), in_initial_frac_infection)
    if "LFTM" == in_model_name:
        return run_dynamic_trial(in_network, LFTM(in_model_param), in_initial_frac_infection)
    if "ICM" == in_model_name:
        return run_dynamic_trial(in_network, ICM(in_model_param), in_initial_frac_infection)

In [ ]:
model_param_range = {"ICM": np.arange(0.05, 1.0, 0.05), "LFTM": np.arange(0.05, 1.0, 0.05), "LATM": np.arange(1, 10, 1)}
model_param_range

In [ ]:
def get_sampled_network_groups():
    sampled_network_groups = {}
    
    data_sources = glob.glob("./Data/*")
    ds_to_files = {os.path.basename(data_src): glob.glob(os.path.join(data_src, "*edges*.csv")) for data_src in data_sources}
    for data_src in ds_to_files:
        for source_data_file in ds_to_files[data_src]:
            data_file_name = os.path.basename(source_data_file)[:-4]
            sampled_network_groups[data_file_name] = {}
            sampledfiles = glob.glob(f"./outputs/{data_file_name}/edges_{data_file_name}_n*.csv")
            num_node_values = set((int(fn.split("_")[-2][1:]) for fn in sampledfiles))
            for num_nodes in num_node_values:
                groupedfiles = glob.glob(f"./outputs/{data_file_name}/edges_{data_file_name}_n{num_nodes}_*.csv")
                sampled_network_groups[data_file_name][num_nodes] = groupedfiles
    
    return sampled_network_groups

In [ ]:
sampled_network_directory = get_sampled_network_groups()

In [ ]:
def get_network(in_network_dataset, in_net_size, in_network_file_path):
    G = nx.read_edgelist(in_network_file_path, delimiter=",", data=False)
    c = nx.clustering(G)
    cc_dist = np.array([c[node] for node in c])
    cc_mean = cc_dist.mean()
    cc_std = cc_dist.std()
    net = {"size": in_net_size,
           "param_name": "n",
           "param": in_net_size,
           "type": in_network_dataset,
           "in_neighbors": nx.to_dict_of_lists(G),
           "cluscoefdist": c,
           "clscoef_mean": cc_mean, 
           "clscoef_std": cc_std }
    return net

In [ ]:
def data_for_network(in_network_dataset, in_net_size, in_net_file_path):
    net = get_network(in_network_dataset, in_net_size, in_net_file_path)
    data = []
    print(f"Starting.. {in_network_dataset} ({in_net_size}) {in_net_file_path}")
    for MODEL_TO_ANALYZE in ["LATM", "LFTM", "ICM"]:
        for model_param in model_param_range[MODEL_TO_ANALYZE]:
            for init_inf in np.arange(0.025, 0.375, 0.025):
                npv, fif = run_model(net, MODEL_TO_ANALYZE, model_param, init_inf)
                data.append([net["type"], net["param"], net["clscoef_mean"], net["clscoef_std"], MODEL_TO_ANALYZE, model_param, init_inf, npv, fif])
    return pd.DataFrame(data, columns=["NetType","NetParam","CC_mean", "CC_std","Model","ModelParam","InitInf", "NPV", "FinalInf"])

In [ ]:
net_params = []
for net_data_set in sampled_network_directory:
    print(net_data_set)
    for net_size in sampled_network_directory[net_data_set]:
        print("\t", net_data_set, net_size, len(sampled_network_directory[net_data_set][net_size]))
        x = [[net_data_set, net_size, file_path] for file_path in sampled_network_directory[net_data_set][net_size]]
        net_params += x
        # print(x)


In [ ]:
with multiprocessing.Pool(multiprocessing.cpu_count() - 1) as P:
    results = P.starmap(data_for_network, net_params)
    
print("DONE")

In [ ]:
df = pd.concat(results, ignore_index=True)

df.to_csv("./cc_outputs/RealData_CCDiffusion.csv", index=False)